In [35]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv("/home/cake/Documents/merged_dataset.csv")

# Selecting columns containing features
features_columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']


def recommend_similar_songs(song_id):
    # Finding the row corresponding to the given song_id
    song_row = data[data['Track ID'] == int(song_id)]
    if song_row.empty:
        print("Song not found.")
        return

    # Extracting features of the selected song
    song_features = song_row[features_columns].values

    # Computing cosine similarity between songs based on features similarity
    similarities = cosine_similarity(data[features_columns], song_features)

    # Adding similarity scores to the dataframe
    data['similarity'] = similarities

    # Get recommended songs based on maximum similarity
    recommended_songs = data.sort_values(by='similarity', ascending=False).head(11)  # Including the song itself

    # Extracting information about the recommended songs
    recommended_songs_info = recommended_songs[['Track ID', 'Title', 'Artist', 'Album', 'Genre', 'similarity']]

    return recommended_songs_info

# Getting user input for the song ID
song_id = input("Enter the Track ID of the song: ")


recommendations = recommend_similar_songs(song_id)
print(recommendations)


     Track ID                            Title                         Artist  \
0         140               queen of the wires  alec k redfearn  the eyesores   
114       831                   late late show                      heres how   
87        797                         joe pico              glove compartment   
88        798   artificial breasts are not new              glove compartment   
118       835                      aint no use                      heres how   
117       834            you showed me the way                      heres how   
104       821                              sir                         hecuba   
77        787  boogerflicking winners go freeh              glove compartment   
116       833              from this moment on                      heres how   
73        783                       puffy onyx              glove compartment   
24        564                the sugar society                       cinwaves   

              Album Genre  

In [40]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors


data = pd.read_csv("/home/cake/Documents/merged_dataset.csv")

# Selecting columns containing features
features_columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14']

def recommend_similar_songs_collaborative(song_id):
    # Initialize NearestNeighbors model
    model = NearestNeighbors(metric='cosine', algorithm='brute')

    # Fitting the model with data (transposed if songs are rows)
    model.fit(data[features_columns])

    # Finding the row corresponding to the given song_id
    song_row_index = data[data['Track ID'] == int(song_id)].index
    if len(song_row_index) == 0:
        print("Song not found.")
        return

    # Getting the feature vector for the given song
    song_features = data.iloc[song_row_index][features_columns]

    # Finding k nearest neighbors
    k_neighbors = 11  # Including the song itself
    distances, indices = model.kneighbors(song_features, n_neighbors=k_neighbors)

    # Extracting information about the recommended songs and their similarity scores
    recommended_songs_info = data.iloc[indices[0]][['Track ID', 'Title', 'Artist', 'Album', 'Genre']]
    similarity_scores = 1 - distances.flatten()  # Cosine similarity scores
    recommended_songs_info['Similarity'] = similarity_scores

    return recommended_songs_info


song_id = input("Enter the Track ID of the song: ")


print("Using Collaborative Filtering:")
recommendations_collab = recommend_similar_songs_collaborative(song_id)
print(recommendations_collab)


Using Collaborative Filtering:
     Track ID                       Title             Artist  \
76        786               electric guru  glove compartment   
81        791  630 encephalitis spraydown  glove compartment   
71        781           shoeshopping high  glove compartment   
140       864                     feather        ian nagoski   
8         533                  mango tree        chandeliers   
85        795       the lower frequencies  glove compartment   
34        743               brittle woman        glass candy   
137       856     last nail in the coffin    i am the liquor   
94        805                    haymaker       guinea worms   
65        775                 nite nurses        glass candy   
60        770                  black veil        glass candy   

                            Album         Genre  Similarity  
76                       quickies          rock    1.000000  
81                       quickies          rock    0.944990  
71            